In [ ]:
!pip uninstall scikit-image-y
!pip install scikit-image==0.16.2
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!pip install efficientnet_pytorch
!pip install openslide-python

In [ ]:
%cd DLMI
%pwd

In [ ]:
from preprocess import dataset_process
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import openslide


In [ ]:
DATA_PATH = '../../input/mvadlmi/'
batch_size = 1

In [ ]:
mode = "train"
data_process = dataset_process(f'{DATA_PATH}', mode, batch_size, shuffle=False, save_tiles = True, num_of_tiles =36, size_of_tile =256)

In [ ]:
num_images_to_display = 1
num_images_displayed = 0 

for idx, (img, target) in tqdm(enumerate(data_process)):
    
    slide = openslide.OpenSlide(target['image_path'][0])
    spacing = 1 / (float(slide.properties['tiff.XResolution']) / 10000)
    image = slide.get_thumbnail(size=(300,200)) 
    cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])
    
    if mode == 'train':
        f,ax =  plt.subplots(1,2 ,figsize=(18,22))
        mask = openslide.OpenSlide(target['mask_path'][0])
        mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
        ax[0].imshow(image) 
        ax[1].imshow(np.asarray(mask_data)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=5) #IMAGE MASKS
        slide.close()       
        
        image_id = target['image_id'][0]
        data_provider = target['data_provider'][0]
        isup_grade = target['isup_grade'][0]
        gleason_score = target['gleason_score'][0]
        ax[0].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score} IMAGE")
        ax[1].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score} IMAGE_MASK")
        plt.show()
        plt.savefig('example.png')
    elif mode == 'test':
        f,ax =  plt.subplots(1,1 ,figsize=(18,22))
        ax[0].imshow(image) 
        slide.close()       
        image_id = target['image_id'][0]
        data_provider = target['data_provider'][0]
        ax[0].set_title(f"ID: {image_id}\nSource: {data_provider} IMAGE")
        plt.show()
        plt.savefig('example1.png')
    num_images_displayed += 1
    if num_images_displayed >= num_images_to_display:
        break
  

In [ ]:
mode = "test"
data_process = dataset_process(f'{DATA_PATH}', mode, batch_size, shuffle=False, save_tiles = True, num_of_tiles=36, size_of_tile=256)

